In [ ]:
!pip uninstall langchain
!pip --default-timeout=100 install -e /Users/shaoxuanzheng/Desktop/langchain 

In [1]:
import os
import yaml
from langchain.agents.agent_toolkits.openapi.spec import ReducedOpenAPISpec, dereference_refs
from langchain.tools import APIOperation, OpenAPISpec

In [2]:
metacube_api_spec = OpenAPISpec.from_file("./opsgpt_kgtools_demo_apis_collection.yml")
search_api_spec = OpenAPISpec.from_file("./opsgpt_search_api.yaml")

Attempting to load an OpenAPI 3.0.1 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.
Attempting to load an OpenAPI 3.0.2 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


In [3]:
metacube_api_spec.servers[0].url = "http://10.0.90.77:7010/api/meta"
search_api_spec.servers[0].url = "http://10.0.70.1:8097"

In [4]:
from langchain.agents.agent_toolkits import NLAToolkit

In [5]:
import openai

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2022-12-01"
os.environ["OPENAI_API_BASE"] = "https://bizseer-opsgpt.openai.azure.com"
os.environ["OPENAI_API_KEY"] = "0a649a1e78a9422494939710468c0fcb"
openai.api_type = "azure"
openai.api_version = "2022-12-01"
openai.api_base = "https://bizseer-opsgpt.openai.azure.com"
openai.api_key = "0a649a1e78a9422494939710468c0fcb"

from langchain.llms import AzureOpenAI

llm = AzureOpenAI(
    deployment_name="base-completion",
    model_name="text-davinci-003"
)

In [6]:
metacube_toolkit = NLAToolkit.from_llm_and_spec(llm=llm, spec=metacube_api_spec, verbose=True)
search_toolkit = NLAToolkit.from_llm_and_spec(llm=llm, spec=search_api_spec, verbose=True)

# from_llm_and_spec -> _get_http_operation_tools -> from_llm_and_method

# methods here:  get
# name='MetaCube.instance__code__get' description="I'm an AI from MetaCube. Instruct what you want, and I'll assist via an API with description: 输入实例编码，即`code`，查询实例的名称、所属对象、创建和更新时间" args_schema=None return_direct=False verbose=False callbacks=None callback_manager=None func=<bound method Chain.run of OpenAPIEndpointChain(memory=None, callbacks=None, callback_manager=None, verbose=True, api_request_chain=APIRequesterChain(memory=None, callbacks=None, callback_manager=None, verbose=True, prompt=PromptTemplate(input_variables=['instructions'], output_parser=APIRequesterOutputParser(), partial_variables={'schema': '/* 输入实例编码，即`code`，查询实例的名称、所属对象、创建和更新时间 */\ntype instance__code__get = (_: {\n/* 实例编码，用于确定唯一的实例 */\n\t\tcode: string,\n}) => any;'}, template='You are a helpful AI Assistant. Please provide JSON arguments to agentFunc() based on the user\'s instructions.\n\nAPI_SCHEMA: ```typescript\n{schema}\n```\n\nUSER_INSTRUCTIONS: "{instructions}"\n\nYour arguments must be plain json provided in a markdown block:\n\nARGS: ```json\n{{valid json conforming to API_SCHEMA}}\n```\n\nExample\nHere is the modified version\n-----\n\nARGS: ```json\n{{"foo": "bar", "baz": {{"qux": "quux"}}}}\n```\n\nThe block must be no more than 1 line long, and all arguments must be valid JSON. All string arguments must be wrapped in double quotes.\nYou MUST strictly comply to the types indicated by the provided schema, including all required args.\n\nIf you don\'t have sufficient information to call the function due to things like requiring specific uuid\'s, you can reply with the following message:\n\nMessage: ```text\nConcise response requesting the additional information that would make calling the function successful.\n```\n\nBegin\n-----\nARGS:\n', template_format='f-string', validate_template=True), llm=AzureOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.completion.Completion'>, model_name='text-davinci-003', temperature=0.7, max_tokens=256, top_p=1, frequency_penalty=0, presence_penalty=0, n=1, best_of=1, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, batch_size=20, request_timeout=None, logit_bias={}, max_retries=6, streaming=False, allowed_special=set(), disallowed_special='all', deployment_name='base-completion'), output_key='text'), api_response_chain=APIResponderChain(memory=None, callbacks=None, callback_manager=None, verbose=True, prompt=PromptTemplate(input_variables=['response', 'instructions'], output_parser=APIResponderOutputParser(), partial_variables={'schema': '/* 输入实例编码，即`code`，查询实例的名称、所属对象、创建和更新时间 */\ntype instance__code__get = (_: {\n/* 实例编码，用于确定唯一的实例 */\n\t\tcode: string,\n}) => any;'}, template='You are a helpful AI assistant trained to answer user queries from API responses.\nYou attempted to call an API, which resulted in:\nAPI_RESPONSE: {response}\nAPI_SCHEMA: {schema}\nUSER_COMMENT: "{instructions}"\n\n\nIf the API_RESPONSE and API_SCHEMA can answer the USER_COMMENT respond with the following markdown json block:\nResponse: ```json\n{{"response": "Human-understandable synthesis of the API_RESPONSE and API_SCHEMA"}}\n```\n\nOtherwise respond with the following markdown json block:\nResponse Error: ```json\n{{"response": "What you did and a concise statement of the resulting error. If it can be easily fixed, provide a suggestion."}}\n```\n\nYou MUST respond as a markdown json code block. The person you are responding to CANNOT see the API_RESPONSE and API_SCHEMA, so if there is any relevant information there you must include it in your response.\n\nBegin:\n---\n', template_format='f-string', validate_template=True), llm=AzureOpenAI(cache=None, verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.completion.Completion'>, model_name='text-davinci-003', temperature=0.7, max_tokens=256, top_p=1, frequency_penalty=0, presence_penalty=0, n=1, best_of=1, model_kwargs={}, openai_api_key=None, openai_api_base=None, openai_organization=None, batch_size=20, request_timeout=None, logit_bias={}, max_retries=6, streaming=False, allowed_special=set(), disallowed_special='all', deployment_name='base-completion'), output_key='text'), api_operation=APIOperation(operation_id='instance__code__get', description='输入实例编码，即`code`，查询实例的名称、所属对象、创建和更新时间', base_url='http://10.0.90.77:7010/api/meta', path='/instance/{code}', method=<HTTPVerb.GET: 'get'>, properties=[APIProperty(name='code', required=True, type='string', default=None, description='实例编码，用于确定唯一的实例', location=<APIPropertyLocation.PATH: 'path'>)], request_body=None), requests=Requests(headers=None, aiosession=None), param_mapping=_ParamMapping(query_params=[], body_params=[], path_params=['code']), return_intermediate_steps=False, instructions_key='instructions', output_key='output', max_text_length=None)> coroutine=None
# check NLATool.from_llm_and_method -> APIOperation.from_openapi_spec -> 需要的信息在operation里
# print("abaaba", metacube_toolkit)

In [7]:
openapi_format_instructions = """Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: what to instruct the AI Action representative.
Observation: The Agent's response
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer. User can't see any of my observations, API responses, links, or tools.
Final Answer: the final answer to the original input question with the right amount of detail

When responding with your Final Answer, remember that the person you are responding to CANNOT see any of your Thought/Action/Action Input/Observations, so if there is any relevant information there you need to include it explicitly in your response."""

In [8]:
from langchain.agents import AgentType, Tool, initialize_agent
mrkl = initialize_agent(
    metacube_toolkit.get_tools() + search_toolkit.get_tools(),
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    agent_kwargs={"format_instructions":openapi_format_instructions}
)
# print(metacube_toolkit.get_tools() + search_toolkit.get_tools())

In [9]:
user_query = "APPCC-0001是什么？"
mrkl.run(user_query)



> Entering new AgentExecutor chain...
 我应该使用MetaCube.instance__code__get来查询APPCC-0001的信息。
Action: MetaCube.instance__code__get
Action Input: APPCC-0001

> Entering new OpenAPIEndpointChain chain...


> Entering new APIRequesterChain chain...
Prompt after formatting:
You are a helpful AI Assistant. Please provide JSON arguments to agentFunc() based on the user's instructions.

API_SCHEMA: ```typescript
/* 输入实例编码，即`code`，查询实例的名称、所属对象、创建和更新时间 */
type instance__code__get = (_: {
/* 实例编码，用于确定唯一的实例 */
		code: string,
}) => any;
```

USER_INSTRUCTIONS: "APPCC-0001"

Your arguments must be plain json provided in a markdown block:

ARGS: ```json
{valid json conforming to API_SCHEMA}
```

Example
-----

ARGS: ```json
{"foo": "bar", "baz": {"qux": "quux"}}
```

The block must be no more than 1 line long, and all arguments must be valid JSON. All string arguments must be wrapped in double quotes.
You MUST strictly comply to the types indicated by the provided schema, including all required args.


> Finished chain.
The USER_COMMENT does not match with the API_RESPONSE and API_SCHEMA. Please check the API documentation for more information about the expected parameters and response.

> Finished chain.

Observation: The USER_COMMENT does not match with the API_RESPONSE and API_SCHEMA. Please check the API documentation for more information about the expected parameters and response.
Thought: APPCC-0001是一个实例编码，它可能代表一个实例，我可以用这个API来查询实例的名称、所属对象、创建和更新时间。
Action: MetaCube.instance__code__get
Action Input: APPCC-0001

> Entering new OpenAPIEndpointChain chain...


> Entering new APIRequesterChain chain...
Prompt after formatting:
You are a helpful AI Assistant. Please provide JSON arguments to agentFunc() based on the user's instructions.

API_SCHEMA: ```typescript
/* 输入实例编码，即`code`，查询实例的名称、所属对象、创建和更新时间 */
type instance__code__get = (_: {
/* 实例编码，用于确定唯一的实例 */
		code: string,
}) => any;
```

USER_INSTRUCTIONS: "APPCC-0001"

Your arguments must be plain json provided in a markdown block:


> Finished chain.
It looks like you may have entered an instance code (APPCC-0001) that does not exist. Please ensure that the instance code is valid and try again.

> Finished chain.

Observation: It looks like you may have entered an instance code (APPCC-0001) that does not exist. Please ensure that the instance code is valid and try again.
Thought: 根据反馈，APPCC-0001不存在，我可以使用MetaCube_Search.search_instance_api_metacube_instance_get来搜索最相关的实例。
Action: MetaCube_Search.search_instance_api_metacube_instance_get
Action Input: APPCC-0001

> Entering new OpenAPIEndpointChain chain...


> Entering new APIRequesterChain chain...
Prompt after formatting:
You are a helpful AI Assistant. Please provide JSON arguments to agentFunc() based on the user's instructions.

API_SCHEMA: ```typescript
/* 实例搜索接口

输入一个实体，实体可能是实例编码或实例名称，返回最相关的实例以及总共相关的实例数量 */
type search_instance_api_metacube_instance_get = (_: {
/* 用于搜索的字符串 */
		query: string,
}) => any;
```

USER_INSTRUCTIONS: "APPCC-0001"

Your arguments mu

'APPCC-0001不是一个有效的实例编码，但是可以搜索到group code为APPCC和instance type code为APPCC的实例，有1346个。'

In [10]:
mrkl.run("查询一下 APPCC-000001 的子图？")



> Entering new AgentExecutor chain...
 可以使用 instance_relation_get API 查询。
Action: MetaCube.instance_relation_get
Action Input: APPCC-000001

> Entering new OpenAPIEndpointChain chain...


> Entering new APIRequesterChain chain...
Prompt after formatting:
You are a helpful AI Assistant. Please provide JSON arguments to agentFunc() based on the user's instructions.

API_SCHEMA: ```typescript
/* 用于查询实例相连的其他实例的列表，以及他们形成子图所对应邻接表的边列表 */
type instance_relation_get = (_: {
/* 实例编码，用于确定唯一的实例 */
		code: string,
/* 实例连接边的方向，默认forward，表明实例是边所连接的源节点(src)，还有可能的取值有reverse，表明实例是边所连接的目标节点(dst)，以及undirected, 表明实例即是边的目标节点，也是源节点，边所连的另一个节点也同时为src和dst */
		direction?: 'forward' | 'reverse' | 'undirected',
}) => any;
```

USER_INSTRUCTIONS: "APPCC-000001"

Your arguments must be plain json provided in a markdown block:

ARGS: ```json
{valid json conforming to API_SCHEMA}
```

Example
-----

ARGS: ```json
{"foo": "bar", "baz": {"qux": "quux"}}
```

The block must be no more than 1 line long, and all argumen

InvalidRequestError: This model's maximum context length is 4097 tokens, however you requested 5951 tokens (5695 in your prompt; 256 for the completion). Please reduce your prompt; or completion length.

In [ ]:
mrkl.run("APPCC-000001和APPCC-000002之间的关系是什么？")

In [42]:
mrkl.run("hipster-kontrast 和 shippingservice是什么关系？")



> Entering new AgentExecutor chain...
 I need to find out the relationship between two instances.
Action: MetaCube.instance_relation_batch_nodes_get
Action Input: hipster-kontrast and shippingservice

> Entering new OpenAPIEndpointChain chain...


> Entering new APIRequesterChain chain...
Prompt after formatting:
You are a helpful AI Assistant. Please provide JSON arguments to agentFunc() based on the user's instructions.

API_SCHEMA: ```typescript
/* 输入多个实例列表，查询这些实例列表之间可能存在的关系、边的列表 */
type instance_relation_batch_nodes_get = (_: {
/* 实例编码列表，用`,`逗号分隔的实例编码 */
		codes: string,
}) => any;
```

USER_INSTRUCTIONS: "hipster-kontrast and shippingservice"

Your arguments must be plain json provided in a markdown block:

ARGS: ```json
{valid json conforming to API_SCHEMA}
```

Example
-----

ARGS: ```json
{"foo": "bar", "baz": {"qux": "quux"}}
```

The block must be no more than 1 line long, and all arguments must be valid JSON. All string arguments must be wrapped in double quotes.
You MUST 

'hipster-kontrast and shippingservice have a native connection, meaning that they are able to communicate with each other.'